# 透過 game_id ，串接 matchinfo 和 kills 檔案處理

In [64]:
import pandas as pd

In [65]:
# 資料 game_id 處理
def game_id_deal(file_name, new_file_name):
    df = pd.read_csv(file_name)
    df['Address'] = df['Address'].str.split('=')
    df['Address'] = df['Address'].str.get(-1)

    df.rename(columns={"Address":"game_id"}, inplace=True)
    df.to_csv(new_file_name)

In [66]:
# kills
kills_origin_name = "/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/kills.csv"
kills_edited_name = "/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/kills_edited.csv"
game_id_deal(kills_origin_name, kills_edited_name)

# matchinfo
matchinfo_origin_name = "/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/matchinfo.csv"
matchinfo_edited_name = "/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/matchinfo_edited.csv"
game_id_deal(matchinfo_origin_name, matchinfo_edited_name)

## 篩選有 Faker 的比賽

In [67]:
df_matchinfo = pd.read_csv("/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/matchinfo_edited.csv")
df_faker = df_matchinfo[['game_id', 'League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag', 'gamelength', 'blueMiddle', 'blueMiddleChamp', 'redMiddle', 'redMiddleChamp']]

In [68]:
# faker_filter
faker_filter = (df_faker.blueMiddle == "Faker") | (df_faker.redMiddle == "Faker")
df_faker = df_faker[faker_filter]

## 抓取 Faker 使用的英雄

In [69]:
# 處理 red, blue 方 faker 使用的 Champ
blueFakerChamp = df_faker[df_faker.blueMiddle == "Faker"]['blueMiddleChamp']
redFakerChamp = df_faker[df_faker.redMiddle == "Faker"]['redMiddleChamp']
fakerChamp = blueFakerChamp.combine_first(redFakerChamp)

In [70]:
# 插入 fakerChamp
df_faker.insert(1, 'fakerChamp', fakerChamp)

## KDA
### 串 kills 檔案，將每一場 Faker 的 KDA 結合到 df_faker 中

In [71]:
df_kills = pd.read_csv("/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/kills_edited.csv")
df_kills = df_kills.drop(df_kills.columns[0], axis=1)

In [72]:
total_kill = []
faker_kill = []
faker_death = []
faker_assist = []

for i in range(len(df_faker)):
    temp_game_id = df_faker.iloc[i].game_id
    # 篩選同場次的紀錄
    game_mask = (df_kills.game_id == temp_game_id)
    df_temp = df_kills[game_mask]
    
    # 整隊擊殺次數
    total_kill_mask = (df_temp.Killer.str.startswith("SKT") == True)
    total_kill_count = len(df_temp[total_kill_mask])
    
    # 擊殺次數
    kill_mask = (df_temp.Killer == "SKT Faker")
    kill_count = len(df_temp[kill_mask])
    
    # 死亡次數
    death_mask = (df_temp.Victim == "SKT Faker")
    death_count = len(df_temp[death_mask])
    
    # 助攻次數
    assist_mask = (df_temp.Assist_1 == "SKT Faker") | (df_temp.Assist_2 == "SKT Faker") | (df_temp.Assist_3 == "SKT Faker") | (df_temp.Assist_4 == "SKT Faker")
    assist_count = len(df_temp[assist_mask])
    
    total_kill.append(total_kill_count)
    faker_kill.append(kill_count)
    faker_death.append(death_count)
    faker_assist.append(assist_count)

In [73]:
df_faker.insert(2, 'fakerKills', faker_kill)
df_faker.insert(3, 'fakerDeaths', faker_death)
df_faker.insert(4, 'fakerAssists', faker_assist)
df_faker.insert(5, 'SKTTotalKills', total_kill)

In [74]:
df_faker.head()

,game_id,fakerChamp,fakerKills,fakerDeaths,fakerAssists,SKTTotalKills,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueMiddle,blueMiddleChamp,redMiddle,redMiddleChamp
2058,156fcf3fa2776e3f,Xerath,1,2,0,1,LCK,2015,Spring,Playoffs,SKT,0,1,CJE,32,Faker,Xerath,Coco,Vladimir
2059,33ad9f88301528e6,Lulu,5,0,5,13,LCK,2015,Spring,Playoffs,CJE,0,1,SKT,31,Coco,Ryze,Faker,Lulu
2060,b817640f2c56dc89,Lulu,4,2,13,20,LCK,2015,Spring,Playoffs,SKT,1,0,CJE,72,Faker,Lulu,Coco,Ziggs
2061,394ba5e889b3e4b5,Leblanc,15,1,10,37,LCK,2015,Spring,Playoffs,SKT,1,0,CJE,35,Faker,Leblanc,Coco,Azir
2065,3c88db0d8da6effd,Xerath,2,4,3,6,LCK,2015,Spring,Season,NJE,1,0,SKT,34,Ggoong,Zed,Faker,Xerath


# Faker 常用英雄分析

In [75]:
fakerUsedChamp = df_faker['fakerChamp'].value_counts()
# 更改欄位名稱
fakerUsedChamp = fakerUsedChamp.rename_axis("Champ").rename("pickCounts")
fakerUsedChamp.to_csv("/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/output_csv/fakerUsedChamp.csv")

In [76]:
df_faker.head()

,game_id,fakerChamp,fakerKills,fakerDeaths,fakerAssists,SKTTotalKills,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueMiddle,blueMiddleChamp,redMiddle,redMiddleChamp
2058,156fcf3fa2776e3f,Xerath,1,2,0,1,LCK,2015,Spring,Playoffs,SKT,0,1,CJE,32,Faker,Xerath,Coco,Vladimir
2059,33ad9f88301528e6,Lulu,5,0,5,13,LCK,2015,Spring,Playoffs,CJE,0,1,SKT,31,Coco,Ryze,Faker,Lulu
2060,b817640f2c56dc89,Lulu,4,2,13,20,LCK,2015,Spring,Playoffs,SKT,1,0,CJE,72,Faker,Lulu,Coco,Ziggs
2061,394ba5e889b3e4b5,Leblanc,15,1,10,37,LCK,2015,Spring,Playoffs,SKT,1,0,CJE,35,Faker,Leblanc,Coco,Azir
2065,3c88db0d8da6effd,Xerath,2,4,3,6,LCK,2015,Spring,Season,NJE,1,0,SKT,34,Ggoong,Zed,Faker,Xerath


## 計算 win rate

In [77]:
# 取前六常玩的 Champ
fakerTopChamp = fakerUsedChamp[:6].to_dict()
fakerTopChamp

{'Orianna': 31,
 'Azir': 31,
 'Ryze': 26,
 'Cassiopeia': 24,
 'Viktor': 23,
 'Lulu': 22}

In [78]:
win_rate = []
for champ, game_count in fakerTopChamp.items():
    # 篩選出 Faker 玩這隻英雄的場次
    champ_mask = (df_faker.fakerChamp == champ)
    temp_df = df_faker[champ_mask]
    
    # 篩選出 Faker 有勝利的場次
    win_mask = ((temp_df.redMiddle == "Faker") & (temp_df.rResult == 1)) | ((temp_df.blueMiddle == "Faker") & (temp_df.bResult == 1))
    temp_df = temp_df[win_mask]
    
    # 計算 win_rate
    win_count = len(temp_df)
    win_rate.append(round(win_count/game_count, 2))
    
for i in range(len(win_rate)):
    win_rate[i] = int(win_rate[i] * 100)
    win_rate[i] = str(win_rate[i]) + "%"
    win_rate[i]
win_rate

['65%', '74%', '85%', '79%', '78%', '73%']

In [79]:
fakerTopChampName = list(fakerTopChamp.keys())
fakerTopChampName

['Orianna', 'Azir', 'Ryze', 'Cassiopeia', 'Viktor', 'Lulu']

In [80]:
amount = list(fakerTopChamp.values())
amount

[31, 31, 26, 24, 23, 22]

## 前六常用英雄之平均 KDA、參團率

In [81]:
KDA = []
TeamfightRate = []
for i in fakerTopChamp:
    game_mask = (df_faker.fakerChamp == i)
    temp_df = df_faker[game_mask]
    
    # KDA
    temp_kill = round(temp_df['fakerKills'].mean())
    temp_death = round(temp_df['fakerDeaths'].mean())
    temp_assist = round(temp_df['fakerKills'].mean())
    temp_KDA = str(temp_kill) + "\\" + str(temp_death) + "\\" + str(temp_assist)
    
    # faker 參團率
    temp_teamfight_rate = round(((temp_df['fakerKills'] + temp_df['fakerAssists']) / temp_df['SKTTotalKills']).mean(), 2)
    
    KDA.append(temp_KDA)
    TeamfightRate.append(temp_teamfight_rate)

In [82]:
KDA

['3\\2\\3', '4\\2\\4', '5\\1\\5', '4\\2\\4', '6\\2\\6', '4\\2\\4']

In [83]:
for i in range(len(TeamfightRate)):
    TeamfightRate[i] = str(int(TeamfightRate[i] * 100)) + "%"
TeamfightRate

['69%', '69%', '66%', '59%', '72%', '78%']

In [84]:
s0 = pd.Series(fakerTopChampName, name="champName")
s1 = pd.Series(win_rate, name="win_rate")
s2 = pd.Series(amount, name="amount")
s3 = pd.Series(KDA, name="mean_KDA")

df_top6 = pd.concat([s0, s1, s2, s3], axis=1)

In [85]:
df_top6

,champName,win_rate,amount,mean_KDA
0,Orianna,65%,31,3\2\3
1,Azir,74%,31,4\2\4
2,Ryze,85%,26,5\1\5
3,Cassiopeia,79%,24,4\2\4
4,Viktor,78%,23,6\2\6
5,Lulu,73%,22,4\2\4


In [86]:
df_top6.to_csv("/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/output_csv/fakerTop6Champ.csv", index=False)

## 同英雄職業選手平均參團率

In [87]:
common_TeamfightRate = []
for i in fakerTopChamp:
    champ_mask = (df_matchinfo.redMiddleChamp == i) | (df_matchinfo.blueMiddleChamp == i)
    df_champ = df_matchinfo[champ_mask]
    df_champ = df_champ[['game_id', 'League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag', 'gamelength', 'blueMiddle', 'blueMiddleChamp', 'redMiddle', 'redMiddleChamp']]
    
    rInfo = df_champ[df_champ.redMiddleChamp == i][['redTeamTag','redMiddle']]
    bInfo = df_champ[df_champ.blueMiddleChamp == i][['blueTeamTag','blueMiddle']]
    champPlayer = rInfo['redMiddle'].combine_first(bInfo['blueMiddle'])
    champTeam = rInfo['redTeamTag'].combine_first(bInfo['blueTeamTag'])
    
    df_champ.insert(1, 'champTeam', champTeam)
    df_champ.insert(2, 'champPlayer', champPlayer)
    
    each_game_TeamfightRate = []
    for i in range(len(df_champ)):
        temp_game = df_champ.iloc[i]
        temp_game_id = temp_game['game_id']
        temp_team = temp_game['champTeam']
        temp_Player = temp_game['champPlayer']
        temp_name = (str(temp_team) + " " + str(temp_Player))
        
        # 去 kills 表中計算 參團率 (kills + assists) / total_kills
        game_mask = (df_kills.game_id == temp_game_id)
        df_temp_kills = df_kills[game_mask]
        # kills
        kills_mask = df_temp_kills.Killer == (temp_name)
        kill_count = len(df_temp_kills[kills_mask])
        # assists
        assists_mask = (df_temp_kills.Assist_1 == temp_name) | (df_temp_kills.Assist_2 == temp_name) | (df_temp_kills.Assist_3 == temp_name) | (df_temp_kills.Assist_4 == temp_name)
        assists_count = len(df_temp_kills[assists_mask])
        # total_kills
        total_kills_mask = (df_temp_kills.Killer.str.startswith(temp_team) == True)
        total_kills_count = len(df_temp_kills[total_kills_mask])
        # 參團率
        if(total_kills_count != 0):
            temp_TeamfightRate = round(((kill_count + assists_count) / total_kills_count), 2)
        else:
            temp_TeamfightRate = 0.0
        each_game_TeamfightRate.append(temp_TeamfightRate)
        
    df_champ.insert(1, 'TeamFightRate', each_game_TeamfightRate)
    
    # 平均參團率
    common_TeamfightRate.append(round(df_champ['TeamFightRate'].mean(), 2))

In [88]:
for i in range(len(common_TeamfightRate)):
    common_TeamfightRate[i] = str(int(common_TeamfightRate[i] * 100)) + "%"
common_TeamfightRate

['70%', '64%', '63%', '65%', '64%', '70%']

In [89]:
s4 = pd.Series(fakerTopChampName , name="champName")
s5 = pd.Series(TeamfightRate, name="FakerTeamfightRate")
s6 = pd.Series(common_TeamfightRate, name="CommonTeamfightRate")

df_TeamfightRate = pd.concat([s4, s5, s6], axis=1)

In [90]:
df_TeamfightRate

,champName,FakerTeamfightRate,CommonTeamfightRate
0,Orianna,69%,70%
1,Azir,69%,64%
2,Ryze,66%,63%
3,Cassiopeia,59%,65%
4,Viktor,72%,64%
5,Lulu,78%,70%


In [91]:
df_TeamfightRate.to_csv("/Users/wuqirong/Documents/大三上課程/資料視覺化/期中報告/archive/output_csv/fakerTeamfightRate.csv", index=False)